# Translation Pipeline Prompt Tuning

This notebook is for testing and tuning the translation prompts using test cases from our assets.

In [1]:
import sys
import os
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd()
sys.path.append(str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

# Import required modules
from src.translation.translator import Translator
from src.prompts.base import TRANSLATION_PROMPT, SYSTEM_PROMPTS, SUPPORTED_LANGUAGES
from src.testing.models import TestArtist  # Use TestArtist instead of Artist
from data.sample_messages import ARTIST_PROMPTS, SAMPLE_MESSAGES, CONVERSATION_THREADS

# Initialize translator
translator = Translator(os.getenv('OPENAI_API_KEY'))

## 1. Test Basic Translation

Test translation of single messages from different artists.

In [ ]:
async def test_basic_translation():
    for artist_name, messages in SAMPLE_MESSAGES.items():
        print(f"\nTesting {artist_name}:")
        for message in messages:
            print(f"Original: {message['text']}")
            artist = TestArtist(
                name=artist_name,
                prompt={'prompt': ARTIST_PROMPTS[artist_name]}
            )
            response = await translator.translate(message['text'], artist, None)
            print(f"Translations: {response.translations}")
            print(f"Metadata: {response.metadata}")

# Run the test
await test_basic_translation()

## 2. Test Context-Aware Translation

Test translation of conversation threads to check context handling.

In [ ]:
from src.translation.schemas import TranslationContext

async def test_context_translation():
    for artist_name, cases in CONVERSATION_THREADS.items():
        print(f"\nTesting {artist_name}:")
        artist = TestArtist(
            name=artist_name,
            prompt={'prompt': ARTIST_PROMPTS[artist_name]}
        )

        context_messages = []
        for thread, msgs in cases.items():
            for i, msg in enumerate(msgs):
                print(f"\nMessage {i+1}:")
                print(f"Context: {msg['context']}")
                print(f"Original: {msg['text']}")
                
                if context_messages:
                    context = TranslationContext(
                        messages=context_messages,
                        current_message=msg['text']
                    )
                    response = await translator.translate_with_context(context, artist)
                else:
                    response = await translator.translate(msg['text'], artist, None)
                
                print("\nTranslations:")
                for lang, trans in response.translations.items():
                    print(f"{lang}: {trans}")
                
                # Add to context for next message
                context_messages.append({"role": "user", "content": msg['text']})
                context_messages.append({
                    "role": "assistant",
                    "content": f"ENG: {response.translations['eng']}\nZH-TW: {response.translations['zh-tw']}"
                })

# Run the test
await test_context_translation()

## 3. Test Safety Checks

Test the safety check system with various content types.

In [ ]:

async def test_safety_checks():
    for artist_name, messages in SAMPLE_MESSAGES.items():
        print(f"\nTesting {artist_name}:")
        for message in messages:
            print(f"Original: {message['text']}")
            artist = TestArtist(
                name=artist_name,
                prompt={'prompt': ARTIST_PROMPTS[artist_name]}
            )
            response = await translator.translate(message['text'], artist, None)
            print(f"Translations: {response.translations}")
            print(f"Metadata: {response.metadata}")
            safety_results = await translator.safety_check(response.translations, artist)
            print(f"Safety results: {safety_results}\n\n")
    
# Run the test
await test_safety_checks()

## 4. Custom Test Cases

Add your own test cases here for specific scenarios.

In [ ]:
async def test_custom_case(artist_name: str, artist_prompt: str, message: str, context: str = None):
    print(f"Testing custom case for {artist_name}:")
    print(f"Context: {context if context else 'None'}")
    print(f"Original: {message}")
    
    artist = Artist(name=artist_name, prompt={'prompt': artist_prompt})
    response = await translator.translate(message, artist, None)
    
    print("\nTranslations:")
    for lang, trans in response.translations.items():
        print(f"{lang}: {trans}")
    print(f"\nMetadata: {response.metadata}")

# Example usage:
# custom_artist_name = "Your Artist"
# custom_artist_prompt = "Your artist prompt"
# custom_message = "Your test message"
# await test_custom_case(custom_artist_name, custom_artist_prompt, custom_message)